In [1]:
import serial #pip3 install pyserial
import time
import subprocess

subprocess.run(['sudo', 'chmod', '777', '/dev/ttyUSB0']) #ttyUSB0 read
port="/dev/ttyUSB0" #(jetson)port="/dev/ttyTHS1" | (pi zero) port="/dev/ttyS0"
ser=serial.Serial(port, baudrate=38400,bytesize=8, parity='N', stopbits=1) #STS servo default baudrate = 1000000

In [5]:
#unlock EPROM, otherwise poweroff reset to default
ID = 2
int_arr = [0xFF, 0xFF, ID, 4, 3, 0x37, 0x00, 0x00]
# update check digit
def update_check_digit(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum +=i
    int_arr[-1] = (checksum & 255)^255 #BitAND then BitXOR

update_check_digit(int_arr)
print('[{}]'.format(', '.join(hex(x) for x in int_arr)))
ser.write(serial.to_bytes(int_arr))
print("send ok")

[0xff, 0xff, 0x2, 0x4, 0x3, 0x37, 0x0, 0xbf]
send ok


In [6]:
#change ID command
NEW_ID = 2
int_arr = [0xFF, 0xFF, ID, 4, 3, 0x05, NEW_ID, 0x00]
# update check digit
def update_check_digit(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum +=i
    int_arr[-1] = (checksum & 255)^255 #BitAND then BitXOR
    
ID = NEW_ID
update_check_digit(int_arr)
print('[{}]'.format(', '.join(hex(x) for x in int_arr)))
ser.write(serial.to_bytes(int_arr))
print("send ok")

[0xff, 0xff, 0x1, 0x4, 0x3, 0x5, 0x2, 0xf0]
send ok


In [6]:
#change baud rate command
baud_rate = 1000000
if baud_rate == 38400:
    baud_rate=7
if baud_rate == 1000000:
    baud_rate=0
int_arr = [0xFF, 0xFF, ID, 4, 3, 0x06, baud_rate, 0x00]
# update check digit
def update_check_digit(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum +=i
    int_arr[-1] = (checksum & 255)^255 #BitAND then BitXOR

update_check_digit(int_arr)
print('[{}]'.format(', '.join(hex(x) for x in int_arr)))
ser.write(serial.to_bytes(int_arr))
print("send ok")

[0xff, 0xff, 0x2, 0x4, 0x3, 0x6, 0x0, 0xf0]
send ok


In [7]:
#change working mode command
# 0 = position mode
# 1 = constant speed rotation mode
# 2 = PWM mode

def change_mod(ID, mode):
    int_arr = [0xFF, 0xFF, ID, 4, 3, 33, mode, 0x00]
    # update check digit
    def update_check_digit(int_arr):
        checksum = 0
        for i in int_arr[2:-1]:
            checksum +=i
        int_arr[-1] = (checksum & 255)^255 #BitAND then BitXOR

    update_check_digit(int_arr)
    print('[{}]'.format(', '.join(hex(x) for x in int_arr)))
    ser.write(serial.to_bytes(int_arr))
    time.sleep(0.1)
    print("send ok")

change_mod(1,0)

[0xff, 0xff, 0x1, 0x4, 0x3, 0x21, 0x0, 0xd6]
send ok


In [1]:
#lock EPROM

int_arr = [0xFF, 0xFF, ID, 4, 3, 0x37, 0x01, 0x00]
# update check digit
def update_check_digit(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum +=i
    int_arr[-1] = (checksum & 255)^255 #BitAND then BitXOR

update_check_digit(int_arr)
print('[{}]'.format(', '.join(hex(x) for x in int_arr)))
ser.write(serial.to_bytes(int_arr))
print("send ok")

NameError: name 'ID' is not defined

In [201]:
from operator import ne
import serial  # pip3 install pyserial
import time

baudrate_STS = 1000000
ID = 2

try:
    port = "/dev/ttyUSB0"
    ser = serial.Serial(
        port, baudrate=baudrate_STS, bytesize=8, parity='N', stopbits=1, timeout=0.05)
except Exception:
    port = "/dev/ttyUSB1"
    ser = serial.Serial(
        port, baudrate=baudrate_STS, bytesize=8, parity='N', stopbits=1, timeout=0.05)

READ_COMM = 0x02
WRITE_COMM = 0x03

MAX_POS_LMT = 0xB # 2 bytes
WORK_MODE = 0x21 # 2 bytes

GOAL_POS = 0x2A   # 2 bytes
CURR_POS = 0x38  # 2 bytes
MOVE_STATUS = 0x42  # 1 bytes

ser.read_all().hex()


def str2arr(newdata_hex, split_strings):
    for index in range(0, len(newdata_hex), 2):
        split_strings.append(int(newdata_hex[index: index + 2], 16))


def hexdata(a):
    print('[{}]'.format(', '.join(hex(x) for x in a)))


def update_check_digit(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum += i
    int_arr[-1] = (checksum & 255) ^ 255  # BitAND then BitXOR
    return int_arr


def calculate_checksum(int_arr):
    checksum = 0
    for i in int_arr[2:-1]:
        checksum += i
    print("checksum" , (checksum & 255) ^ 255 , int_arr[-1])
    return (checksum & 255) ^ 255 == int_arr[-1]

def change_mod( mode):
    if mode == 0:
        int_arr = [0xFF, 0xFF, ID, 4, 3, 33, mode, 0x00]
        send(int_arr)
        int_arr = [0xFF, 0xFF, ID, 5, 3, 11, 0xff, 0xf, 0x00]
        send(int_arr)
        print("ID ", ID, " change mode", mode, "ok")
    if mode == 3:
        int_arr = [0xFF, 0xFF, ID, 4, 3, 33, mode, 0x00]
        send(int_arr)
        int_arr = [0xFF, 0xFF, ID, 5, 3, 11, 0x00, 0x00, 0x00]
        send(int_arr)
        print("ID ", ID, " change mode", mode, "ok")

def read():
    split_strings = []
    newdata_hex = get_data()
    str2arr(newdata_hex, split_strings)
    hexdata(split_strings[1:21])
    hexdata(split_strings[21:41])
    hexdata(split_strings[41:61])
    hexdata(split_strings[61:81])
    hexdata(split_strings[81:])
    print(len(split_strings))
    if calculate_checksum(split_strings) == True:
        del split_strings[0:5]
        read_work_mode = split_strings[WORK_MODE]
        read_max_pos_mlt = split_strings[MAX_POS_LMT]  | split_strings[MAX_POS_LMT+1]  << 8
        read_goal_pos = split_strings[GOAL_POS]  | split_strings[GOAL_POS+1]  << 8
        read_curr_pos = split_strings[CURR_POS]  | split_strings[CURR_POS+1]  << 8
        read_move_status = split_strings[MOVE_STATUS]
    print(read_work_mode, read_max_pos_mlt, read_goal_pos, read_curr_pos, read_move_status)
    return read_work_mode, read_max_pos_mlt, read_goal_pos, read_curr_pos, read_move_status


def get_data():
    newdata_hex = ""
    #read_arr = [0xff, 0xff, ID, 0x4, 0x2, 0x2a, 0x1e, 0x00]
    read_arr = [0xff, 0xff, ID, 0x4, READ_COMM, 0, 87, 0x00]

    while not newdata_hex.startswith("ffff") and len(newdata_hex) < 72:
        ser.write(serial.to_bytes(update_check_digit(read_arr)))
        time.sleep(0.06)
        newdata_hex = ser.read_all().hex()
        #print("newdata_hex_get",newdata_hex, len(newdata_hex))
    print(newdata_hex, len(newdata_hex))
    return newdata_hex

def send(int_arr):
    newdata_hex = ""
    int_arr = update_check_digit(int_arr)
    while not newdata_hex.startswith("ffff") and len(newdata_hex) < 12:
        ser.write(serial.to_bytes(int_arr))
        time.sleep(0.06)
        newdata_hex = ser.read_all().hex()
        #print("newdata_hex_send",newdata_hex, len(newdata_hex))
    return newdata_hex

a,b,c,d,e = read()
print(a,b,c,d,e)
change_mod(3)
a,b,c,d,e = read()
print(a,b,c,d,e)
change_mod(0)
a,b,c,d,e = read()
print(a,b,c,d,e)



ffff02590003080009020200000100000000465a28e8036c262620200000000000e8030100000314c8500ac8c80000000000000000e8030000000000010000000000004c17000000fb0f0000000000000000000000010a320196fa05ff 186
[0xff, 0x2, 0x59, 0x0, 0x3, 0x8, 0x0, 0x9, 0x2, 0x2, 0x0, 0x0, 0x1, 0x0, 0x0, 0x0, 0x0, 0x46, 0x5a, 0x28]
[0xe8, 0x3, 0x6c, 0x26, 0x26, 0x20, 0x20, 0x0, 0x0, 0x0, 0x0, 0x0, 0xe8, 0x3, 0x1, 0x0, 0x0, 0x3, 0x14, 0xc8]
[0x50, 0xa, 0xc8, 0xc8, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0xe8, 0x3, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1]
[0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x4c, 0x17, 0x0, 0x0, 0x0, 0xfb, 0xf, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0]
[0x0, 0x0, 0x0, 0x0, 0x1, 0xa, 0x32, 0x1, 0x96, 0xfa, 0x5, 0xff]
93
checksum 255 255
3 0 0 0 0
3 0 0 0 0
ID  2  change mode 3 ok
ffff02590003080009020200000100000000465a28e8036c262620200000000000e8030100000314c8500ac8c80000000000000000e8030000000000010000000000004c17000000fb0f0000000000000000000000010a320196fa05ff 186
[0xff, 0x2, 0x59, 0x0, 0x3, 0x8, 0x0, 0x9, 0x2, 0x2, 0x0, 0x0, 0

In [72]:
import time

def timer(delay):
    start_time = time.time()
    time_diff = 0
    while time_diff < delay:
        a, b, c = svo1.ser.read()
        print("read_goal_pos = ", a,
              "read_pres_pos = ", b,
              "read_move_status = ", c)
        time_diff = time.time() - start_time

help=range(10000)
print(help)
while help:
    del help[:1000]

print(help)

range(0, 10000)


TypeError: 'range' object does not support item deletion

In [144]:
import random
 
# using random.sample()
# to generate random number list
res = random.sample(range(1, 99), 1000)
 
# printing result
print ("Random number list is : " +  str(res))

ValueError: Sample larger than population or is negative

In [179]:
import time

start_time = time.time()
time_diff = 0
mylist = [0xff, 0xff, 0x2, 0x59, 0x0, 0x3, 0x8, 0x0, 0x9, 0x2, 0x2, 0x0, 0x0, 0x1, 0x0, 0x0, 0xff, 0xf, 0x46, 0x5a, 0x28, 0xe8, 0x3, 0x6c, 0x26, 0x26, 0x20, 0x20, 0x0, 0x0, 0x0, 0x0, 0x0, 0xe8, 0x3, 0x1, 0x0, 0x0, 0x0, 0x14, 0xc8, 0x50, 0xa, 0xc8, 0xc8, 0x1, 0x0, 0xff, 0x3, 0x0, 0x0, 0xfa, 0x0, 0xe8, 0x3, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1, 0xfc, 0x3, 0x0, 0x0, 0x18, 0x4, 0x4b, 0x18, 0x0, 0x0, 0x0, 0xff, 0x3, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1, 0xa, 0x32, 0x1, 0x96, 0xfa, 0x5, 0xe4]
del mylist[0:5]
time_diff = time.time() - start_time
print(time_diff)
print(mylist)

0.0008332729339599609
[3, 8, 0, 9, 2, 2, 0, 0, 1, 0, 0, 255, 15, 70, 90, 40, 232, 3, 108, 38, 38, 32, 32, 0, 0, 0, 0, 0, 232, 3, 1, 0, 0, 0, 20, 200, 80, 10, 200, 200, 1, 0, 255, 3, 0, 0, 250, 0, 232, 3, 0, 0, 0, 0, 0, 1, 252, 3, 0, 0, 24, 4, 75, 24, 0, 0, 0, 255, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 50, 1, 150, 250, 5, 228]


In [172]:
import time

start_time = time.time()
time_diff = 0
mylist = [0xff, 0xff, 0x2, 0x59, 0x0, 0x3, 0x8, 0x0, 0x9, 0x2, 0x2, 0x0, 0x0, 0x1, 0x0, 0x0, 0xff, 0xf, 0x46, 0x5a, 0x28, 0xe8, 0x3, 0x6c, 0x26, 0x26, 0x20, 0x20, 0x0, 0x0, 0x0, 0x0, 0x0, 0xe8, 0x3, 0x1, 0x0, 0x0, 0x0, 0x14, 0xc8, 0x50, 0xa, 0xc8, 0xc8, 0x1, 0x0, 0xff, 0x3, 0x0, 0x0, 0xfa, 0x0, 0xe8, 0x3, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1, 0xfc, 0x3, 0x0, 0x0, 0x18, 0x4, 0x4b, 0x18, 0x0, 0x0, 0x0, 0xff, 0x3, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x1, 0xa, 0x32, 0x1, 0x96, 0xfa, 0x5, 0xe4]
for i in range(5):
    mylist.remove(mylist[0])
time_diff = time.time() - start_time
print(time_diff)
print(mylist)

0.0007650852203369141
[3, 8, 0, 9, 2, 2, 0, 0, 1, 0, 0, 255, 15, 70, 90, 40, 232, 3, 108, 38, 38, 32, 32, 0, 0, 0, 0, 0, 232, 3, 1, 0, 0, 0, 20, 200, 80, 10, 200, 200, 1, 0, 255, 3, 0, 0, 250, 0, 232, 3, 0, 0, 0, 0, 0, 1, 252, 3, 0, 0, 24, 4, 75, 24, 0, 0, 0, 255, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 50, 1, 150, 250, 5, 228]
